In [1]:
import string
import random
from typing import List
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline


比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [2]:
def create_random_data(row_num:int)->List[tuple]:
     result = list()
     a_str = string.ascii_uppercase
     for i in range(row_num):
         random_letter = random.choice(a_str)
         result.append((random_letter,random.randint(1,row_num),random.random()))
     return result

In [3]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)

    spark = session_factory.build_session()
    columns = ["age","name","address"]
    df = spark.createDataFrame(data,columns).cache()
    df.toPandas().head(5)

session_factory_normal = DemoSQLSessionFactory(name="normal")
%timeit test_performance(session_factory_normal)

3.29 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%timeit test_performance(session_factory_arrow)

3.05 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
